In [4]:
import xarray as xr
sat_20 = xr.open_dataset('/data/acker/WashU_V5_NA/V5NA04.02.HybridPM25.xNorthAmerica.2015001-2015364.nc').sel(lat=slice(10,72), lon=slice(-180.,-50)).to_dataframe()
sat_20.reset_index(inplace=True)
sat_20

,lon,lat,GWRPM25
0,-169.995,14.005,NaN
1,-169.995,14.015,NaN
2,-169.995,14.025,NaN
3,-169.995,14.035,NaN
4,-169.995,14.045,NaN
...,...,...,...
63719995,-52.005,67.955,NaN
63719996,-52.005,67.965,NaN
63719997,-52.005,67.975,NaN
63719998,-52.005,67.985,NaN


In [14]:
import xarray as xr
import pandas as pd
from shapely.geometry import Point
import geopandas as gpd

#Import uncertainty nc files as dfs
data = xr.open_dataset('/data/acker/WashU_NA_unc/V5NA04.02.HybridPM25EFull.xNorthAmerica.2015001-2015364.nc', autoclose=True)
#data_sel = data.sel(lat=slice(6.5,85.5), lon=slice(-180.,-19.5))
unc_20 = data.to_dataframe()
data = xr.open_dataset('/data/acker/WashU_NA_unc/V5NA04.02.HybridPM25EFull.xNorthAmerica.2016001-2016364.nc', autoclose=True)
#data_sel = data.sel(lat=slice(6.5,85.5), lon=slice(-180.,-19.5))
unc_21 = data.to_dataframe()
data = xr.open_dataset('/data/acker/WashU_NA_unc/V5NA04.02.HybridPM25EFull.xNorthAmerica.2017001-2017364.nc', autoclose=True)
#data_sel = data.sel(lat=slice(6.5,85.5), lon=slice(-180.,-19.5))
unc_22 = data.to_dataframe()

#Import satellite data
data = xr.open_dataset('/data/acker/WashU_V5_NA/V5NA04.02.HybridPM25.xNorthAmerica.2015001-2015364.nc', autoclose=True)
#data_sel = data.sel(lat=slice(6.5,85.5), lon=slice(-180.,-19.5))
sat_20 = data.to_dataframe()
data = xr.open_dataset('/data/acker/WashU_V5_NA/V5NA04.02.HybridPM25.xNorthAmerica.2016001-2016364.nc', autoclose=True)
#data_sel = data.sel(lat=slice(6.5,85.5), lon=slice(-180.,-19.5))
sat_21 = data.to_dataframe()
data = xr.open_dataset('/data/acker/WashU_V5_NA/V5NA04.02.HybridPM25.xNorthAmerica.2017001-2017364.nc', autoclose=True)
#data_sel = data.sel(lat=slice(6.5,85.5), lon=slice(-180.,-19.5))
sat_22 = data.to_dataframe()

#Start merging, calulating uncertainty in percent, and filtering out >90% uncertainty

# 2020
merged_20 = pd.merge(sat_20, unc_20, left_index=True, right_index=True, how='inner')
merged_20.reset_index(inplace=True)
merged_20['percent'] = ((merged_20['GWRPM25SIGMAFULL'] / merged_20['GWRPM25']) * 100)
filtered_df_20 = merged_20[merged_20['GWRPM25SIGMAFULL'] < 90]
#print(filtered_df_20)

#2021
merged_21 = pd.merge(sat_21, unc_21, left_index=True, right_index=True, how='inner')
merged_21.reset_index(inplace=True)
merged_21['percent'] = ((merged_21['GWRPM25SIGMAFULL'] / merged_21['GWRPM25']) * 100)
filtered_df_21 = merged_21[merged_21['GWRPM25SIGMAFULL'] < 90]
#print(filtered_df_21)

#2022
merged_22 = pd.merge(sat_22, unc_22, left_index=True, right_index=True, how='inner')
merged_22.reset_index(inplace=True)
merged_22['percent'] = ((merged_22['GWRPM25SIGMAFULL'] / merged_22['GWRPM25']) * 100)
filtered_df_22 = merged_22[merged_22['GWRPM25SIGMAFULL'] < 90]
#print(filtered_df_22)

# Combine all filtered dataframes, groupby location, find mean, and export as csv
combined_df = pd.concat([filtered_df_20, filtered_df_21, filtered_df_22])
mean_df = combined_df.groupby(['lat', 'lon'])['GWRPM25'].mean().reset_index()

print('coverting to shapefile')
mean_df['geometry'] = mean_df.apply(lambda row: Point(row['lon'], row['lat']), axis=1)
df_gpd = gpd.GeoDataFrame(mean_df, geometry='geometry', crs="EPSG:4269")

#print(mean_df)

#data_20_21 = pd.merge(filtered_df_20, filtered_df_21, left_index=True, right_index=True, how = 'inner')
#all_data = pd.merge(data_20_21, filtered_df_22, left_index=True, right_index=True, how = 'inner')
#all_data
df_gpd

coverting to shapefile


,lat,lon,GWRPM25,geometry
0,14.005,-91.455,29.433334,POINT (-91.455 14.005)
1,14.005,-91.445,29.500000,POINT (-91.445 14.005)
2,14.005,-91.435,29.433334,POINT (-91.435 14.005)
3,14.005,-91.425,29.300001,POINT (-91.425 14.005)
4,14.005,-91.415,29.199999,POINT (-91.415 14.005)
...,...,...,...,...
24904708,67.995,-64.735,0.366667,POINT (-64.735 67.995)
24904709,67.995,-64.725,0.366667,POINT (-64.725 67.995)
24904710,67.995,-64.715,0.366667,POINT (-64.715 67.995)
24904711,67.995,-64.705,0.366667,POINT (-64.705 67.995)


In [23]:
df_gpd.to_file('/data/acker/test.shp')

In [1]:
import xarray as xr
import pandas as pd
from shapely.geometry import Point
import geopandas as gpd

#Import satellite data
data = xr.open_dataset('/data/acker/WashU_V5_NA/V5NA04.02.HybridPM25.xNorthAmerica.2015001-2015364.nc', autoclose=True)
#data_sel = data.sel(lat=slice(6.5,85.5), lon=slice(-180.,-19.5))
sat_20 = data.to_dataframe()
sat_20.reset_index(inplace=True)

data = xr.open_dataset('/data/acker/WashU_V5_NA/V5NA04.02.HybridPM25.xNorthAmerica.2016001-2016364.nc', autoclose=True)
#data_sel = data.sel(lat=slice(6.5,85.5), lon=slice(-180.,-19.5))
sat_21 = data.to_dataframe()
sat_21.reset_index(inplace=True)

data = xr.open_dataset('/data/acker/WashU_V5_NA/V5NA04.02.HybridPM25.xNorthAmerica.2017001-2017364.nc', autoclose=True)
#data_sel = data.sel(lat=slice(6.5,85.5), lon=slice(-180.,-19.5))
sat_22 = data.to_dataframe()
sat_22.reset_index(inplace=True)

#Start merging, calulating uncertainty in percent, and filtering out >90% uncertainty

# Combine all filtered dataframes, groupby location, find mean, and export as csv
combined_df = pd.concat([sat_20, sat_21, sat_22])
mean_df = combined_df.groupby(['lat', 'lon'])['GWRPM25'].mean().reset_index()

print('coverting to shapefile')
mean_df['geometry'] = mean_df.apply(lambda row: Point(row['lon'], row['lat']), axis=1)
df_gpd = gpd.GeoDataFrame(mean_df, geometry='geometry', crs="EPSG:4269")

#print(mean_df)

#data_20_21 = pd.merge(filtered_df_20, filtered_df_21, left_index=True, right_index=True, how = 'inner')
#all_data = pd.merge(data_20_21, filtered_df_22, left_index=True, right_index=True, how = 'inner')
#all_data
df_gpd

coverting to shapefile


,lat,lon,GWRPM25,geometry
0,14.005,-169.995,NaN,POINT (-169.995 14.005)
1,14.005,-169.985,NaN,POINT (-169.985 14.005)
2,14.005,-169.975,NaN,POINT (-169.975 14.005)
3,14.005,-169.965,NaN,POINT (-169.965 14.005)
4,14.005,-169.955,NaN,POINT (-169.955 14.005)
...,...,...,...,...
63719995,67.995,-52.045,NaN,POINT (-52.045 67.995)
63719996,67.995,-52.035,NaN,POINT (-52.035 67.995)
63719997,67.995,-52.025,NaN,POINT (-52.025 67.995)
63719998,67.995,-52.015,NaN,POINT (-52.015 67.995)


In [ ]:
df_gpd.to_file('/data/acker/test2.shp')

/home/acker/miniconda3/envs/tempo/lib/python3.12/site-packages/pyogrio/raw.py:709: RuntimeWarning: 2GB file size limit reached for /data/acker/test2.dbf. Going on, but might cause compatibility issues with third party software
  ogr_write(


: 

In [21]:
data = xr.open_dataset('/data/acker/WashU_GL_unc/V5GL04.HybridPM25E.NorthAmerica.202001-202012.nc', autoclose=True)
data = data.to_dataframe()
data

GWRPM25SIGMA
lon         lat                    
-131.994995 25.004999           NaN
            25.014999           NaN
            25.025000           NaN
            25.035000           NaN
            25.045000           NaN
...                             ...
-65.004997  54.955002           1.5
            54.965000           1.4
            54.974998           1.4
            54.985001           1.4
            54.994999           1.4

[20100000 rows x 1 columns]

In [22]:
data['GWRPM25SIGMA'].mean()

2.6179519

In [25]:
import xarray as xr
data = xr.open_dataset('/data/acker/WashU_NA_unc/V5NA04.02.HybridPM25EFull.xNorthAmerica.2017001-2017364.nc', autoclose=True)
#data_sel = data.sel(lat=slice(6.5,85.5), lon=slice(-180.,-19.5))
unc_20 = data.to_dataframe()
unc_20

GWRPM25SIGMAFULL
lon      lat                     
-169.995 14.005               NaN
         14.015               NaN
         14.025               NaN
         14.035               NaN
         14.045               NaN
...                           ...
-52.005  67.955               NaN
         67.965               NaN
         67.975               NaN
         67.985               NaN
         67.995               NaN

[63720000 rows x 1 columns]

In [ ]:
data

<xarray.Dataset> Size: 255MB
Dimensions:           (lon: 11800, lat: 5400)
Coordinates:
  * lon               (lon) float64 94kB -170.0 -170.0 -170.0 ... -52.02 -52.01
  * lat               (lat) float64 43kB 14.01 14.02 14.03 ... 67.97 67.98 68.0
Data variables:
    GWRPM25SIGMAFULL  (lat, lon) float32 255MB nan nan nan nan ... nan nan nan
Attributes:
    Title:            Hybrid PM_2_._5' Full [\mug/m^3]
    Contact:          Aaron van Donkelaar <aaron.vandonkelaar@wustl.edu>
    References:       van Donkelaar et al., Monthly Global Estimates of Fine ...
    Filename:         ./RegionalGWR-NA-BiWeekly-Geo20230823-GM20230905-V20240...
    Format:           NetCDF-4
    Delta_Lat:        0.01
    Delta_Lon:        0.01
    SpatialCoverage:  xNorthAmerica
    Start_Date:       2017001
    End_Date:         2017364

: 

In [18]:
unc_20.reset_index()

,lon,lat,GWRPM25SIGMAFULL
0,-169.995,14.005,NaN
1,-169.995,14.015,NaN
2,-169.995,14.025,NaN
3,-169.995,14.035,NaN
4,-169.995,14.045,NaN
...,...,...,...
63719995,-52.005,67.955,NaN
63719996,-52.005,67.965,NaN
63719997,-52.005,67.975,NaN
63719998,-52.005,67.985,NaN


In [19]:
unc_20['GWRPM25SIGMAFULL'].mean()

43.55832

In [20]:
unc_20[unc_20['GWRPM25SIGMAFULL'] >= 90]

GWRPM25SIGMAFULL
lon      lat                     
-159.945 59.945         93.099998
         59.955         90.800003
-159.665 22.115         91.599998
         22.125         93.800003
-159.655 22.115         92.099998
...                           ...
-61.585  66.785         96.599998
         66.795         98.099998
-61.575  66.775         96.500000
         66.795         90.599998
-61.565  66.765         90.500000

[129358 rows x 1 columns]